In [1]:
from turtle import title
import requests
import pandas as pd
from bs4 import BeautifulSoup

from pandas_datareader import data as wb
import datetime
import matplotlib.pyplot as plt

In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
import yfinance as yf
yf.pdr_override()

data = wb.get_data_yahoo("003490.KS", start="2022-05-01", end="2022-11-30")

[*********************100%***********************]  1 of 1 completed


In [4]:
df = data
keyword = ['대한항공']
date_0 = []
date_1 = []
title = []

In [5]:
df['Price'] = 0
for i in range(0, len(data)-1):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

df.to_csv('stock_data.csv')

price_data = pd.read_csv('stock_data.csv')
df_0 = price_data[price_data['Price']==0]['Date']

for i in range(0,len(df_0)):
    date_0.append(str(df_0.tolist()[i])[:10].replace('-',''))

df_1 = price_data[price_data['Price']==1]['Date']

for i in range(0,len(df_1)):
    date_1.append(str(df_1.tolist()[i])[:10].replace('-',''))

<ipython-input-5-8f699da9836e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 1
<ipython-input-5-8f699da9836e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 0


In [6]:
def news_save(dates):
    for dates in dates:
        
        year = dates[:4]
        month = dates[4:6]
        day = dates[6:8]

        dates_ = year+'.'+month+'.'+day


        URL = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_word}&sort=1&photo=0&field=0&pd=3&ds={startdate}&de={enddate}&cluster_rank=37&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{start_date}to{end_date},a:all&start={page_num}'

        url = URL.format(search_word = keyword,startdate= dates_,enddate = dates_,
                                        start_date=dates,end_date = dates,page_num = '1')

        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
                'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
        response = requests.get(url, headers = headers)

        soup = BeautifulSoup(response.text,'lxml')
        articles = soup.find_all('div', {'class' : 'news_area'})
        

        for article in articles:
            article_title = article.find('a', attrs = {'class' : 'news_tit'})['title']
            title.append(article_title)

In [7]:
news_save(date_0)

title_df_2 = pd.DataFrame(title, columns=['뉴스제목'])
title_df_2['주가변동'] = 0

title = []
error_cnt = 0

news_save(date_1)

title_df_3 = pd.DataFrame(title, columns=['뉴스제목'])
title_df_3['주가변동'] = 1

title_df2 = pd.concat([title_df_2, title_df_3])
title_df2.to_csv('news_data.csv', index=False, encoding='utf-8')

title_df2


,뉴스제목,주가변동
0,"우기홍 대한항공 사장 ""항공업 50위까지 추락…비효과적 PCR 고집해야하나""",0
1,"이스라엘관광청, ""성지순례 및 프리미엄 여행 개발··· 직항노선 재개 강력 요청""",0
2,"KOTRA, 대한항공과 중소·중견기업 항공 운송 지원 업무협약 체결",0
3,항공·정유 '스마일'…전자업계는 '울상',0
4,"한진家 편법 증여에 세금 140억 철퇴... 법원 ""부과 정당""",0
...,...,...
635,"영국, 대한항공-아시아나 시정안 수용…합병 승인 가능성 커져",1
636,"[데일리안 오늘 뉴스 종합] 남욱 ""곽상도, 김만배에게"", 尹, 與지도부 만찬서 ""...",1
637,"英, 대한항공-아시아나 합병 시정조치안 수용… 사실상 승인",1
638,"英, 대한항공·아시아나 합병 사실상 승인",1
